# Assignment 4 

In [1]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 39 kB/s 
     |████████████████████████████████| 198 kB 45.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=c6f6a6ff8ed0fb6bb39fd912158dd19f7ad05d9a0b96185c60a52f50dadb06c7
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


Creating a SparkContext

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, IntegerType, DateType
from pyspark.sql.functions import col, asc,desc
import json
import csv
from io import StringIO
from pyspark.sql import HiveContext
sc = SparkContext()

Installing Hbase and JDK and setting up Java Home system variable

In [3]:
pip install happybase

     |████████████████████████████████| 40 kB 3.1 MB/s 
     |████████████████████████████████| 361 kB 7.6 MB/s 
     |████████████████████████████████| 49 kB 5.8 MB/s 
  Created wheel for happybase: filename=happybase-1.2.0-py2.py3-none-any.whl size=26623 sha256=b532a992bb12a2c268cef65cd20faf77629873a13e028d9fef95f28894e9d810
  Stored in directory: /root/.cache/pip/wheels/66/63/48/437f79a3724a4a529830f87d766a7d34228d623e845de2c321
  Created wheel for thriftpy2: filename=thriftpy2-0.4.14-cp37-cp37m-linux_x86_64.whl size=944460 sha256=07d01940e10ab3ef3cce94c168412569f045dec8a3c97827b78eb800ca1dc7ec
  Stored in directory: /root/.cache/pip/wheels/2a/f5/49/9c0d851aa64b58db72883cf9393cc824d536bdf13f5c83cff4
Successfully built happybase thriftpy2


In [4]:
!apt-get update > /dev/null

In [5]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [6]:
!wget https://archive.apache.org/dist/hbase/2.4.9/hbase-2.4.9-bin.tar.gz

--2022-04-21 22:01:03--  https://archive.apache.org/dist/hbase/2.4.9/hbase-2.4.9-bin.tar.gz
Resolving archive.apache.org (archive.apache.org)... 138.201.131.134, 2a01:4f8:172:2ec5::2
Connecting to archive.apache.org (archive.apache.org)|138.201.131.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 283496242 (270M) [application/x-gzip]
Saving to: ‘hbase-2.4.9-bin.tar.gz’

hbase-2.4.9-bin.tar 100%[===================>] 270.36M  8.71MB/s    in 24s     

2022-04-21 22:01:27 (11.5 MB/s) - ‘hbase-2.4.9-bin.tar.gz’ saved [283496242/283496242]



In [7]:
!tar xzf hbase-2.4.9-bin.tar.gz

In [8]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Running Thrift server and HMaster of Hbase

In [9]:
!/content/hbase-2.4.9/bin/stop-hbase.sh

no hbase master found


In [10]:
!/content/hbase-2.4.9/bin/hbase-daemon.sh start thrift

running thrift, logging to /content/hbase-2.4.9/bin/../logs/hbase--thrift-9de666796d1d.out


In [11]:
!/content/hbase-2.4.9/bin/start-hbase.sh

running master, logging to /content/hbase-2.4.9/bin/../logs/hbase--master-9de666796d1d.out


In [12]:
!jps

152 SparkSubmit
1705 Jps
1627 HMaster
1420 ThriftServer


In [13]:
import happybase as hb


In [14]:

def connection_and_create_in_hbase(flag):
  try:
    connection = hb.Connection()
    connection.open()
   
    if flag:
      connection.create_table('Fake_data',{'cf':dict(), 
                                          'cf':dict(), 
                                          'cf':dict(),
                                          'cf':dict(),
                                          'cf':dict(),
                                          'cf':dict(),
                                          'cf':dict(),
                                          'cf':dict()}) 
    return connection
  except Exception as e:
    print(e)
    connection.close()

In [17]:
def store_csv_into_hbase():
  try:
    connection = connection_and_create_in_hbase(True)
    
    table = connection.table('Fake_data')
    
    csvreader = csv.reader(open('/content/Fake_data.csv'))
   
    row_count = 0
   
    for row in csvreader:
       
        row_count += 1
        if row_count == 1:
            pass
        elif row_count == 10001:
            break
        else:
            table.put(row[0], { "cf:Birth_Country": str(row[1]), "cf:Email": row[2], "cf:First_Name": row[3],"cf:Income": row[4], "cf:Job": row[5], "cf:Last_name": row[6],"cf:Loan_Approved": row[7], "cf:SSN": row[8] })
                   
  

  except Exception as e:
    print(e)
  finally:
     
      connection.close()

In [21]:
store_csv_into_hbase()

Printing 10 rows in the table

In [22]:

connection = connection_and_create_in_hbase(False)
table = connection.table('Fake_data')

rows = table.rows([str(i) for i in range(10)])
for key, data in rows:
    print(key, data)

b'0' {b'cf:Birth_Country': b'Bosnia and Herzegovina', b'cf:Email': b'emily15@whitehead.com', b'cf:First_Name': b'Melissa', b'cf:Income': b'109957', b'cf:Job': b'Telecommunications researcher', b'cf:Last_name': b'Miranda', b'cf:Loan_Approved': b'False', b'cf:SSN': b'129-41-7773'}
b'1' {b'cf:Birth_Country': b'Belgium', b'cf:Email': b'ronald87@yahoo.com', b'cf:First_Name': b'Curtis', b'cf:Income': b'301884', b'cf:Job': b'Animal nutritionist', b'cf:Last_name': b'Garrett', b'cf:Loan_Approved': b'True', b'cf:SSN': b'212-74-3976'}
b'2' {b'cf:Birth_Country': b'United Kingdom', b'cf:Email': b'hannah29@gmail.com', b'cf:First_Name': b'Connor', b'cf:Income': b'341584', b'cf:Job': b'English as a foreign language teacher', b'cf:Last_name': b'Steele', b'cf:Loan_Approved': b'False', b'cf:SSN': b'024-35-3834'}
b'3' {b'cf:Birth_Country': b'Kiribati', b'cf:Email': b'derrick59@hotmail.com', b'cf:First_Name': b'Adam', b'cf:Income': b'448293', b'cf:Job': b'Surveyor, commercial/residential', b'cf:Last_name':

In [ ]:
!/content/hbase-2.4.9/bin/hbase shell

Streaming output truncated to the last 5000 lines.
                      21-13-3429                                                
 9718                 column=cf1:Birth_Country, timestamp=2022-04-21T02:16:22.70
                      6, value=Eritrea                                          
 9718                 column=cf2:Email, timestamp=2022-04-21T02:16:22.706, value
                      =joseph41@gmail.com                                       
 9718                 column=cf3:First_Name, timestamp=2022-04-21T02:16:22.706, 
                      value=Janet                                               
 9718                 column=cf4:Income, timestamp=2022-04-21T02:16:22.706, valu
                      e=277267                                                  
 9718                 column=cf5:Job, timestamp=2022-04-21T02:16:22.706, value=M
                      erchant navy officer                                      
 9718                 column=cf6:Last_Name, timestamp=2022

Creating a SparkSession and DataFrame of CSv file 

In [23]:

spark = SparkSession(sc)
df=spark.read.format("csv").option("header","true").load("/content/Fake_data.csv")

In [24]:
df.show()

+---+--------------------+--------------------+----------+------+--------------------+---------+-------------+-----------+
|_c0|       Birth_Country|               Email|First_Name|Income|                 Job|Last_name|Loan_Approved|        SSN|
+---+--------------------+--------------------+----------+------+--------------------+---------+-------------+-----------+
|  0|Bosnia and Herzeg...|emily15@whitehead...|   Melissa|109957|Telecommunication...|  Miranda|        False|129-41-7773|
|  1|             Belgium|  ronald87@yahoo.com|    Curtis|301884| Animal nutritionist|  Garrett|         True|212-74-3976|
|  2|      United Kingdom|  hannah29@gmail.com|    Connor|341584|English as a fore...|   Steele|        False|024-35-3834|
|  3|            Kiribati|derrick59@hotmail...|      Adam|448293|Surveyor, commerc...|   Newman|        False|157-82-4486|
|  4|            Malaysia|wendycarpenter@wa...|     Jared| 53621|   Drilling engineer|     Mann|         True|199-56-2824|
|  5|           

Saving the DataFrame as a HiveMetastore Table

In [25]:
# import pyspark.sql.DataFrameWriter.saveAsTable
df.write.mode("overwrite").saveAsTable("fake_data_tbl")

## 1. Find birth country which has highest amount of people

In [26]:
spark.sql("select count(Birth_Country) as Count, Birth_Country from fake_data_tbl group by Birth_Country order by 1 DESC ").show(1)

+-----+-------------+
|Count|Birth_Country|
+-----+-------------+
|   91|        Korea|
+-----+-------------+
only showing top 1 row



##  2. Find average income of people who are born in united states of america

In [27]:
spark.sql("""select AVG(t.Income), t.Birth_Country from fake_data_tbl t 
group by t.Birth_country
having LOWER(t.Birth_Country) == LOWER('united states of america')""").show()

+------------------+--------------------+
|       avg(Income)|       Birth_Country|
+------------------+--------------------+
|208759.82352941178|United States of ...|
+------------------+--------------------+



 ## 3. How many people has income over 100,000 but their loan is not approved

In [28]:
spark.sql(" select count(*) Nos_of_People from fake_data_tbl as t where t.income > 100000 and Loan_Approved == False").show()

+-------------+
|Nos_of_People|
+-------------+
|         4009|
+-------------+



## 4. Find top 10 people with highest income in United States of america. (Print their names, income and jobs)

In [29]:
spark.sql("Select t.First_Name || ' ' || t.Last_name Name, t.Income, t.Job from fake_data_tbl as t where LOWER(t.Birth_Country) == LOWER('united states of america') order by INT(Income) DESC  ").show(10)
# spark.sql("SELECT tb.First_Name,tb.Last_name, tb.Job, tb.Income  FROM Fake_data_tbl tb where tb.Birth_Country = 'United States of America' order by tb.Income DESC Limit 10 ").show()

+----------------+------+--------------------+
|            Name|Income|                 Job|
+----------------+------+--------------------+
|   Alyssa Miller|482588|Amenity horticult...|
|    Hunter Walls|468946|Psychologist, pri...|
|  Rose Henderson|426115|Adult guidance wo...|
|Danielle Leonard|389810|Furniture conserv...|
|     Terry Klein|380410|       Meteorologist|
|    Cindy Newton|370322|Research scientis...|
|  Scott Mitchell|368913|       Art therapist|
|Christy Sandoval|355150|      Engineer, land|
|  Kelly Reynolds|341448|           Press sub|
|  Kristina Smith|338804|           Herbalist|
+----------------+------+--------------------+
only showing top 10 rows



In [30]:
spark.sql("SELECT tb.First_Name,tb.Last_name, tb.Job, tb.Income  FROM Fake_data_tbl tb where tb.Birth_Country = 'United States of America' order by INT(tb.Income) DESC Limit 10 ").show()

+----------+---------+--------------------+------+
|First_Name|Last_name|                 Job|Income|
+----------+---------+--------------------+------+
|    Alyssa|   Miller|Amenity horticult...|482588|
|    Hunter|    Walls|Psychologist, pri...|468946|
|      Rose|Henderson|Adult guidance wo...|426115|
|  Danielle|  Leonard|Furniture conserv...|389810|
|     Terry|    Klein|       Meteorologist|380410|
|     Cindy|   Newton|Research scientis...|370322|
|     Scott| Mitchell|       Art therapist|368913|
|   Christy| Sandoval|      Engineer, land|355150|
|     Kelly| Reynolds|           Press sub|341448|
|  Kristina|    Smith|           Herbalist|338804|
+----------+---------+--------------------+------+



In [31]:
spark.sql("SELECT count(*) FROM fake_data_tbl tb ").show()

+--------+
|count(1)|
+--------+
|   10000|
+--------+



## 5. How many number of distinct jobs are there?

In [32]:
spark.sql("SELECT count(DISTINCT(tb.Job)) FROM fake_data_tbl tb").show()

+-------------------+
|count(DISTINCT Job)|
+-------------------+
|                639|
+-------------------+



## 6. How many writers earn less than 100,000?



In [33]:
spark.sql("SELECT count(*) FROM fake_data_tbl tb where tb.Income <100000 and tb.Job = 'Writer' ").show()

+--------+
|count(1)|
+--------+
|       5|
+--------+



References:
https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrameWriter.saveAsTable.html#pyspark.sql.DataFrameWriter.saveAsTable



https://spark.apache.org/docs/2.2.0/sql-programming-guide.html#:~:text=Spark%20SQL%20is%20a%20Spark,information%20to%20perform%20extra%20optimizations


https://happybase.readthedocs.io/en/latest/user.html


https://happybase.readthedocs.io/en/latest/api.html

https://sparkbyexamples.com/apache-hbase-tutorial/

https://www.youtube.com/watch?v=CsYaTIUvez0

https://www.rittmanmead.com/blog/2015/05/loading-updating-and-deleting-from-hbase-tables-using-hiveql-and-python/


Trying to create an external Table in Hive which is associated with HBase table. But gives an error.

In [35]:


spark.sql("""
CREATE EXTERNAL TABLE hbase_hive_fake_data_tbl
 (key string,
  Birth_Country string,
  Email string,
  First_Name string,
  Income INT,
  Job string,
  Last_name   string,
  Loan_Approved string,
  SSN string
) 
STORED BY 'org.apache.hadoop.hive.hbase.HBaseStorageHandler'
WITH SERDEPROPERTIES('hbase.columns.mapping' = ':_c0,cf1:Birth_Country,cf2:Email,cf3:First_Name,cf4:Income,cf5:Job,cf6:Last_name,cf7:Loan_Approved,cf8:SSN') TBLPROPERTIES ('hbase.table.name' = 'Fake_data');""")

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 206))



ParseException: ignored

In [38]:
!/content/hbase-2.4.9/bin/hbase shell

2022-04-21 22:07:33,781 WARN  [main] util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
HBase Shell
Use "help" to get list of supported commands.
Use "exit" to quit this interactive shell.
For Reference, please visit: http://hbase.apache.org/2.0/book.html#shell
Version 2.4.9, rc49f7f63fca144765bf7c2da41791769286dfccc, Fri Dec 17 19:02:09 PST 2021
Took 0.0065 seconds                                                             
hbase:001:0> list
TABLE                                                                           
Fake_data                                                                       
1 row(s)
Took 0.6189 seconds                                                             
=> ["Fake_data"]
hbase:002:0> ^C


In [36]:
spark.sql("SHOW TABLES").show()

+---------+-------------+-----------+
|namespace|    tableName|isTemporary|
+---------+-------------+-----------+
|  default|fake_data_tbl|      false|
+---------+-------------+-----------+

